In [2]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.vgg import VGG16_BN_Weights
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

PyTorch Version:  2.0.1
Torchvision Version:  0.15.2


In this cell we define some important parameters which we`ll use in this process.

In [3]:
# Top level data directory. Here we assume the format of the directory conforms 
#   to the ImageFolder structure
data_dir = "../../dataset/Plant_leave_diseases_dataset_without_augmentation"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "vgg"

# Number of classes in the dataset
num_classes = 39

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for 
num_epochs = 2

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = True

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("The selected device is:", device)

The selected device is: cpu


### **Helper functions**

#### **Code for training and validating the model**

The `train_model` function receives a PyTorch model (`model`), a dictionary of dataloaders (`dataloaders`), a loss function (`criterion`), an optimizer (`optimizer`), the number of epochs we'll train for (`num_epochs`) and a boolean flag for when the model is the Inception v3 (`is_inception`).

This function trains the model for the specified number of epochs and after each epoch, it passes the validation set through the network (validation step). It also keeps track of the best performing model (in terms of validation accuracy), and at the end of training it returns this best performing model. After each epoch, the training and validation accuracies are printed.

In [4]:
def train_model(model, dataloaders, criterion, optimizer, device, working_mode, num_epochs=25):
    since = time.time()

    val_acc_history = []
    val_loss_history = []
    train_acc_history = []
    train_loss_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # temporary variables to store predictions and true labels
        epoch_preds = []
        epoch_true = []

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # In train mode we calculate the loss by summing the final output and the auxiliary output
                    # but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()


                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # store predictions and true labels
                # we're only interested in validation performance
                if phase == 'val':  
                    epoch_preds.extend(preds.tolist())
                    epoch_true.extend(labels.data.tolist())



            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} ---> {} Loss: {:.4f} Acc: {:.4f}'.format(working_mode, phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_preds, best_true = epoch_preds, epoch_true

            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)
            else:
                train_acc_history.append(epoch_acc)
                train_loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('{} ---> Training complete in {:.0f}m {:.0f}s'.format(working_mode, time_elapsed // 60, time_elapsed % 60))
    print('{} ---> Best val Acc: {:4f}'.format(working_mode, best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_preds, best_true, val_acc_history, val_loss_history, train_acc_history, train_loss_history

#### **Code for properly seting model parameter's `.requires_grad` attribute**

This cell defines a function that acts accordingly wheter we are finetunning the whole model or only using it as a feature extractor. If we are finetunning the whole model, it is ok to update all the weights of the model. But, if we are only using the convolutional network as a fixed feature extractor, we don't want to update the weights of the model at all.

In [5]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

### **Initializing and Reshaping the network**
Firstly, the fundamental aspect of Transfer learning is the reshaping of the network in order to make it suit the dataset of the problem that we want to solve. This being said, we can consider that most of the pretrained models we'll work with have been pretrained in ImageNet. I.e, a dataset with 1000 classes.

So, independently if we will fine tune the whole model, or only use it as a fixed feature extractor, we need to reshape it's last layer. Considering it was trained for ImageNet, it is intuitive to think that this last layer is a fully conected layer that has 1000 output nodes. Maybe, it also has an Softmax layer with 1000 inputs and 1000 outputs.

In order that the reshaping works as expected, we need to make this final layers have the same number of input nodes that before (it comes from the internal structure of the network) and it must have a number of output nodes that is exactly the same of the number of classes in our dataset.

In [6]:
def initialize_model(num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    # variables is model specific.
    model_ft = None
    input_size = 0

    # Defining the model as VGG:
    if use_pretrained:
        model_ft = models.vgg16_bn(weights=VGG16_BN_Weights.DEFAULT)
    else:
        model_ft = models.vgg16_bn()
    
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224

    return model_ft, input_size

In [7]:
# Initialize the model for this run
use_pretrained_bool = True
feature_extract_bool = True

model_ft, input_size = initialize_model(num_classes, feature_extract_bool, use_pretrained=use_pretrained_bool)
# Print the model we just instantiated
print(model_ft)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

### **Load Data**

In [8]:
def plot_labels_histogram(dataloader):
    all_labels = []
    for _, labels in tqdm(dataloader):
        all_labels.extend(labels.numpy())
        
    plt.hist(all_labels, bins=len(set(all_labels))) # number of bins equals the number of unique labels
    plt.xlabel('Label')
    plt.ylabel('Frequency')
    plt.title('Histogram of labels in DataLoader')
    plt.show()

In [9]:
def stratified_split(dataset):
    train_idx, valid_idx= train_test_split(np.arange(len(dataset.targets)),
                                        test_size=0.2,
                                        shuffle=True,
                                        stratify=dataset.targets
                                        )
    


    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

    dataloaders_dict = {'train': train_loader, 'val': valid_loader}

    return dataloaders_dict




def regular_split(train_percent, dataset):
    # Determine the lengths of training and validation sets
    train_len = int(train_percent * len(dataset))  # 80% for training
    val_len = len(dataset) - train_len   # 20% for validation

    # Split the dataset
    train_set, val_set = random_split(dataset, [train_len, val_len])

    # Create a dictionary of the datasets
    image_datasets = {'train': train_set, 'val': val_set}

    # Create the dataloaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=False, num_workers=4) for x in ['train', 'val']}

    return dataloaders_dict

Now that we know the input size that the network takes, we need to transform our data so it fits this input layer.

In [10]:
def load_data(data_dir, input_size, batch_size, train_percent=0.8):
    # Define your transform
    transform = transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(data_dir, transform=transform)

    total_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    #plot_labels_histogram(total_dataloader)

    dataloaders_dict = regular_split(train_percent, dataset)

    #plot_labels_histogram(dataloaders_dict['train'])

    #plot_labels_histogram(dataloaders_dict['val'])


    return dataloaders_dict

In [11]:
# Load the data
dataloaders_dict = load_data(data_dir, input_size, batch_size, train_percent=0.8)

### **Create the optimizer**

Now that the model structure is correct, we only need to create an optimizer tha will only update the desired parameters.

When deciding whether we wanted only to use the CNN as a fixed feature extractor or to fine tune it, we've set the parameters `.requires_grad` attribute accordingly.

So now we know that all the parameteres with *`.requires_grad` == True* must be updated. Next, we make a list of such parameters and input this list to the SGD algorithm constructor.

In [12]:
def define_optimizer(model_ft, device, feature_extract):
    # Send the model to GPU
    model_ft = model_ft.to(device)

    # Gather the parameters to be optimized/updated in this run. If we are
    # finetuning we will be updating all parameters. However, if we are
    # doing feature extract method, we will only update the parameters
    # that we have just initialized, i.e. the parameters with requires_grad
    # is True. Notice that, when finetuning, all the parameters have requires_grad = True.
    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
    return optimizer_ft

In [13]:
# Define the optimizer
optimizer_ft = define_optimizer(model_ft, device, feature_extract_bool)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


### **Running training and validation routines**

In [14]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Train and evaluate
# In order to produce matrics for the model, we will store confusion matrix necessary values.
model_ft, best_preds, best_true, val_acc_hist, val_loss_hist, train_acc_hist, train_loss_hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, device, "teste", num_epochs=num_epochs)

Epoch 0/14
----------


KeyboardInterrupt: 

### **Saving model and training statistics**

In [ ]:
# Save the model
model_path = os.path.join(current_dir, f"{working_mode}/model.pth")
torch.save(model_ft.state_dict(), model_path)

# Save the validation accuracy history
val_acc_hist_np = np.array([h.item() for h in val_acc_hist])
val_acc_hist_path = os.path.join(current_dir, f"{working_mode}/val_acc_history.csv")
np.savetxt(val_acc_hist_path, val_acc_hist_np, delimiter=",")

#Save the validation loss history
val_loss_hist_np = np.array([h for h in val_loss_hist])
val_loss_hist_path = os.path.join(current_dir, f"{working_mode}/val_loss_history.csv")
np.savetxt(val_loss_hist_path, val_loss_hist_np, delimiter=",")


# Save the train accuracy history
train_acc_hist_np = np.array([h.item() for h in train_acc_hist])
train_acc_hist_path = os.path.join(current_dir, f"{working_mode}/train_acc_history.csv")
np.savetxt(train_acc_hist_path, train_acc_hist_np, delimiter=",")

#Save the train loss history
train_loss_hist_np = np.array([h for h in train_loss_hist])
train_loss_hist_path = os.path.join(current_dir, f"{working_mode}/train_loss_history.csv")
np.savetxt(train_loss_hist_path, train_loss_hist_np, delimiter=",")

### **Plotting training statistics**

In [ ]:
def train_val_plot(pathTrain, pathVal, title, stat):
    # Read the CSV file
    dfVal = pd.read_csv(pathVal)
    dfTrain = pd.read_csv(pathTrain)

    plt.figure(figsize=(6, 5))
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(stat)
    plt.plot(dfVal, label='Validation ' + stat)
    plt.plot(dfTrain, label='Training ' + stat)
    plt.legend(ncol=4, bbox_to_anchor=(0.5,-0.5), loc='lower center', edgecolor='w')
    plt.tight_layout()
    plt.savefig(f"{title} - {stat}.png")
    plt.show()

In [ ]:
train_val_plot(train_loss_hist_path, val_loss_hist_path, 'VGG16 as a fixed feature extractor', 'Loss')
train_val_plot(train_acc_hist_path, val_acc_hist_path, 'VGG16 as a fixed feature extractor', 'Accuracy')

### **Understanding what the model see's (Heatmap)**

Using the library, we will follow the steps:
1. Choose a random image from each one of the 38 image classes.
2. Load the images with OpenCV
3.  For each loaded image:
    1. Make a forward pass to find the category index with the highest score and compute intermediate activations.
    2. Make the visualization.

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2

In [ ]:
def preprocess_image(img):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    preprocessing = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
    return preprocessing(img.copy()).unsqueeze(0)

def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

In [ ]:
IMAGE_PATH = "../../dataset/Plant_leave_diseases_dataset_without_augmentation/Apple___Black_rot/image (35).JPG"

In [ ]:
target_layers = [model_ft.features[-1]]

img = cv2.imread(IMAGE_PATH, 1)
img = np.float32(img) / 255
# Opencv loads as BGR:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_img = preprocess_image(img)

grad_cam = GradCAM(model=model_ft, target_layers=target_layers, use_cuda=True)

target_category = None

grayscale_cam = grad_cam(input_img, target_category)

grayscale_cam = cv2.resize(grayscale_cam, (img.shape[1], img.shape[0]))
cam = show_cam_on_image(img, grayscale_cam)

cv2.imwrite("cam.jpg", cam)

### **Plotting GradCam**

In [ ]:
fig, ax = plt.subplots(1,2)
img = cv2.imread(IMAGE_PATH)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cam = cv2.imread("./cam.jpg")
cam = cv2.cvtColor(cam, cv2.COLOR_BGR2RGB)
ax[0].imshow(img)
ax[1].imshow(cam);